In [45]:
# PERM

import random
import numpy as np

def PERM(L, C_plus = 10, C_minus = 1):
    """
    Generates a self-avoiding walk (SAW) of length L using PERM.

    The walk starts at the origin (0,0). [haven't write]
    """
    # print("new run")
    # Special case: L = 0
    if L == 0:
        return 1

    # Thresholds
    Z_hat = [1] * (L + 1)  # Z_hat[i] is the estimate of Z for length i, initialised to 1 for all i in [0, L]
    W_plus = lambda n: C_plus * Z_hat[n]
    W_minus = lambda n: C_minus * Z_hat[n]

    # Initialise a stack: (path, total weight, total walks)
    stack = [([(0,0)], 1, 0)]  # Start at the origin with weight 1 and length 0
    total_weights = np.zeros(L + 1)  # Stores the total weight of each length, initialised to 0
    total_walks = np.zeros(L + 1)  # Stores the total number of walks of each length, initialised to 0

    while stack: # While there are still items in the stack
        # Pop the last item from the stack
        path, weight, n = stack.pop()
        # print(f"Popped: {path}, weight: {weight}, n: {n}")
        if n == L: # Reached the target length
            # total_weights[L] += weight # Add the weight to the total weight for length L
            # total_walks[L] += 1 # Increment the total number of walks for length L
            # print("Reached target length")
            continue # Next iteration
        
        # Not reached the target length yet, so we need to generate valid moves
        last_pos = path[-1] # Get the last position in the path
        valid_moves = []
        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]: # Possible moves (Right, Left, Up, Down)
            # Check if the new position is valid (not visited)
            new_pos = (last_pos[0] + dx, last_pos[1] + dy)
            if new_pos not in path:
                valid_moves.append(new_pos)
        w_n = len(valid_moves) # Number of valid moves
        if w_n == 0:
            # print("Dead end")
            continue  # Attrition, move to next iteration
        
        # Valid moves is not 0, update weight
        new_weight = weight * w_n
        new_path = path + [random.choice(valid_moves)]  # Add a new move to the path
        
        # Pruning/enrichment
        if new_weight < W_minus(n+1): # Add 1 because of 0-indexing
            # Weight too low, prune with probability 0.5
            if np.random.rand() < 0.5:
                # print("Pruned")
                continue  # Prune and move to the next iteration
            else:
                new_weight *= 2 # Survives pruning, balance out the weight of pruned paths
        elif new_weight > W_plus(n+1): # Good path, enrich it
            k = 2 # Total number of copies
            for _ in range(k):
                stack.append((new_path.copy(), new_weight / k, n+1)) # Add the new path to the stack k times
                # Update Z_hat for both copies
                total_walks[n+1] += 1
                total_weights[n+1] += new_weight / k
                Z_hat[n+1] = total_weights[n+1] / total_walks[n+1] if total_walks[n+1] > 0 else 0
                # print("Enriched")
                # print("total_weights", total_weights[L])
                # print("total_walks", total_walks[L])
                # print("Z_hat", Z_hat[L])
            continue # Continue to the next iteration
        
        # Didn't prune or enrich, just add the new path to the stack
        stack.append((new_path, new_weight, n+1))
        # Update Z_hat
        total_walks[n+1] += 1
        total_weights[n+1] += new_weight
        Z_hat[n+1] = total_weights[n+1] / total_walks[n+1] if total_walks[n+1] > 0 else 0
        # print("total_weights", total_weights[L])
        # print("total_walks", total_walks[L])
        # print("Z_hat", Z_hat[L])
    
    # print("Final total_weights", total_weights[L])
    return total_weights[L]

# Compute Monte Carlo estimates for c_L for L = 0 to 10 using 100,000 samples per length
for i in range(11):
    samples = [PERM(i) for _ in range(1000000)]
    # print("Samples:", samples)
    print(f"Estimated c_{i}: {np.mean(samples)}")

Estimated c_0: 0.0
Estimated c_1: 4.0
Estimated c_2: 12.0
Estimated c_3: 36.0
Estimated c_4: 100.001502
Estimated c_5: 284.003847
Estimated c_6: 779.966118
Estimated c_7: 2171.990619
Estimated c_8: 5915.143908
Estimated c_9: 16268.539071375
Estimated c_10: 44113.20750253125
